Peakpower 30KWp and Losses of 15%

In [326]:
import pvlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# The next line is to make the plots appear in the notebook
%matplotlib inline 

### Getting data using PVLIB

In [327]:
#input parameters for retriving weather data hourly base

########  USER INPUTS FOR THE STREAMLIT APP #######

lat = 52.41
long = 9.71
location_name = "Hannover"  
altitude = 55.0
timezone = 'Etc/GMT-1'
pv_tilt = 35.0

inverter_name = 'ABB__MICRO_0_25_I_OUTD_US_208__208V_'  # 
module_name = 'Canadian_Solar_CS5P_220M___2009_'
temp_model_par_name = 'open_rack_glass_glass'  #'open_rack_glass_glass', 'close_mount_glass_glass', 
                                               #'open_rack_glass_polymer','insulated_back_glass_polymer
                #Source: https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.temperature.sapm_cell.html
        
surface_azimuth_angle = 0 # Azimuth angle of the module surface. North=0, East=90, South=180, West=270.
peakpowervar = 10 #Nominal power of the PV system, in kW
pvtechmaterial = 'crystSi' # PV technology. Choices are: "crystSi", "CIS", "CdTe" and "Unknown".
mountingtype = 'free'   ''' Type of mounting of the PV modules. Choices are: "free" for 
                            free-standing and "building" for building-integrated. 
                        '''    
loss = 15.0      #Sum of system losses, in percent.
sun_tracker = 0 # Type of suntracking used, 0=fixed, 1=single horizontal axis aligned north-south, 
                # 2=two-axis tracking, 3=vertical axis tracking, 4=single horizontal axis aligned east-west, 
                # 5=single inclined axis aligned north-south.
                
########--------##############

coordinates = [
    (lat, long, location_name, altitude, timezone) 
]

weather = pvlib.iotools.get_pvgis_tmy(lat, long, map_variables=True)
pvlib_weather = weather[0]
pvlib_weather

,temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure
time(UTC),,,,,,,,,
2006-01-01 00:00:00+00:00,6.05,87.47,0.0,0.0,0.0,311.46,6.42,222.0,99233.0
2006-01-01 01:00:00+00:00,5.58,89.12,0.0,0.0,0.0,314.57,6.06,219.0,99249.0
2006-01-01 02:00:00+00:00,5.11,90.77,0.0,0.0,0.0,317.67,5.69,216.0,99266.0
2006-01-01 03:00:00+00:00,4.65,92.42,0.0,0.0,0.0,320.77,5.33,215.0,99306.0
2006-01-01 04:00:00+00:00,4.18,94.07,0.0,0.0,0.0,323.88,4.96,215.0,99345.0
...,...,...,...,...,...,...,...,...,...
2006-12-31 19:00:00+00:00,8.39,79.23,0.0,0.0,0.0,295.95,8.24,219.0,101075.0
2006-12-31 20:00:00+00:00,7.92,80.88,0.0,0.0,0.0,299.05,7.88,218.0,100945.0
2006-12-31 21:00:00+00:00,7.46,82.53,0.0,0.0,0.0,302.15,7.51,216.0,100704.0


# Obtaing data for energy production

In [328]:
#retriving data for Solar Position
tmys = []
solposdf = pd.DataFrame()
'''
pv_tilt = 35.0
inverter_name = 'ABB__MICRO_0_25_I_OUTD_US_208__208V_'
module_name = 'Canadian_Solar_CS5P_220M___2009_'
temp_model_par_name = 'open_rack_glass_glass'
surface_azimuth_angle = 0 # Azimuth angle of the module surface. North=0, East=90, South=180, West=270.
'''       
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')


module = sandia_modules[module_name]  

inverter = sapm_inverters[inverter_name]

temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm'][temp_model_par_name]

#creating a dict with the input parameters
system = {'module': module, 
          'inverter': inverter,
          'surface_azimuth': surface_azimuth_angle}

# Making the call to retrive data

for location in coordinates:
    latitude, longitude, name, altitude, timezone = location
    weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude, map_variables=True)[0]
    weather.index.name = "utc_time"
    tmys.append(weather)

for location, weather in zip(coordinates, tmys):
    latitude, longitude, name, altitude, timezone = location
    system['surface_tilt'] = pv_tilt # Tilt angle of the module surface. Up=0, horizon=90.
    solpos = pvlib.solarposition.get_solarposition(
        time=weather.index,
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        temperature=weather["temp_air"],
        pressure=pvlib.atmosphere.alt2pres(altitude)
    )
    solposdf = solpos
solposdf.head(2)   

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
utc_time,,,,,,
2006-01-01 00:00:00+00:00,149.846384,149.846384,-59.846384,-59.846384,16.428874,-3.313169
2006-01-01 01:00:00+00:00,145.421016,145.421016,-55.421016,-55.421016,41.019742,-3.332981


In [329]:
for location, weather in zip(coordinates, tmys):
    latitude, longitude, name, altitude, timezone = location
    system['surface_tilt'] = pv_tilt,   # Tilt angle of the module surface. Up=0, horizon=90.
    map_variables=True,
    dni_extra = pvlib.irradiance.get_extra_radiation(weather.index)
    airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
    pressure = pvlib.atmosphere.alt2pres(altitude)
    am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
    aoi = pvlib.irradiance.aoi(
        system['surface_tilt'],
        system['surface_azimuth'],
        solpos["apparent_zenith"],
        solpos["azimuth"],
    )
       
    total_irradiance = pvlib.irradiance.get_total_irradiance(
        system['surface_tilt'],
        system['surface_azimuth'],
        solpos['apparent_zenith'],
        solpos['azimuth'],
        weather['dni'],
        weather['ghi'],
        weather['dhi'],
        dni_extra=dni_extra,
        model='haydavies',
    )
    cell_temperature = pvlib.temperature.sapm_cell(
        total_irradiance['poa_global'],
        weather["temp_air"],
        weather["wind_speed"],
        **temperature_model_parameters,
    )
    effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
        total_irradiance['poa_direct'],
        total_irradiance['poa_diffuse'],
        am_abs,
        aoi,
        module,
    )
    dc = pvlib.pvsystem.sapm(effective_irradiance, cell_temperature, module)
    ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)

In [330]:
#sandia_modules

In [331]:
#print(type(sandia_modules))

In [332]:
#sapm_inverters

In [333]:
#print(type(sapm_inverters))

In [334]:
#tmp_mode = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']

#'open_rack_glass_glass', 'close_mount_glass_glass', 'open_rack_glass_polymer','insulated_back_glass_polymer

In [335]:
#print(type(tmp_mode))

## Converting all variables to DataFrames

In [336]:
dni_extra = dni_extra.to_frame(name="dni_PVlib")
airmass = airmass.to_frame(name="airmass")
am_abs = am_abs.to_frame(name="am_abs")
aoi = aoi.to_frame(name="aoi")
cell_temperature = cell_temperature.to_frame(name="cell_temperature")
effective_irradiance = effective_irradiance.to_frame(name="effective_irradiance")
ac = ac.to_frame(name="ac_current")

## Joining all dataframes in one big dataframe

In [337]:
#joining everything.

pv_berlin = pvlib_weather.join([solposdf,aoi, 
                                total_irradiance, cell_temperature, 
                                effective_irradiance, ac, dc], how="left", sort =True)
    
pv_berlin.sort_values(by="time(UTC)").head(2)

,temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure,apparent_zenith,...,cell_temperature,effective_irradiance,ac_current,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx
time(UTC),,,,,,,,,,,,,,,,,,,,,
2006-01-01 00:00:00+00:00,6.05,87.47,0.0,0.0,0.0,311.46,6.42,222.0,99233.0,149.846384,...,6.05,0.0,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-01 01:00:00+00:00,5.58,89.12,0.0,0.0,0.0,314.57,6.06,219.0,99249.0,145.421016,...,5.58,0.0,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Obtaining the target value (power values) from PVGIS will be used. 

In [338]:
import io
import json
from pathlib import Path
import requests
from pvlib.iotools import read_epw, parse_epw
import warnings
from pvlib._deprecation import pvlibDeprecationWarning
import datetime

URL = 'https://re.jrc.ec.europa.eu/api/v5_2/'
'''
##### VARS
peakpowervar = 30 #Nominal power of the PV system, in kW
pvtechmaterial = 'crystSi' # PV technology. Choices are: "crystSi", "CIS", "CdTe" and "Unknown".
mountingtype = 'free'    #Type of mounting of the PV modules. Choices are: "free" for free-standing and "building" for building-integrated.
loss = 15      #Sum of system losses, in percent.
sun_tracker = 0     Type of suntracking used, 0=fixed, 1=single horizontal axis aligned north-south, 
                    2=two-axis tracking, 3=vertical axis tracking, 4=single horizontal axis aligned east-west, 
                    5=single inclined axis aligned north-south.
'''
# Dictionary mapping PVGIS names to pvlib names
VARIABLE_MAP = {
    'G(h)': 'ghi',
    'Gb(n)': 'dni',
    'Gd(h)': 'dhi',
    'G(i)': 'poa_global',
    'Gb(i)': 'poa_direct',
    'Gd(i)': 'poa_sky_diffuse',
    'Gr(i)': 'poa_ground_diffuse',
    'H_sun': 'solar_elevation',
    'T2m': 'temp_air',
    'RH': 'relative_humidity',
    'SP': 'pressure',
    'WS10m': 'wind_speed',
    'WD10m': 'wind_direction',
}


def get_pvgis_hourly(latitude, longitude, start=None, end=None,
                     raddatabase=None, components=True,
                     surface_tilt=0, surface_azimuth=0,
                     outputformat='json',
                     usehorizon=True, userhorizon=None,
                     pvcalculation=False,
                     peakpower= peakpowervar, 
                     pvtechchoice= pvtechmaterial,
                     mountingplace= mountingtype, 
                     loss=0, 
                     trackingtype=0,
                     optimal_surface_tilt=False, optimalangles=False,
                     url=URL, map_variables=True, timeout=60):
        
    # noqa: E501
    # use requests to format the query string by passing params dictionary
    params = {'lat': latitude, 'lon': longitude, 'outputformat': outputformat,
              'angle': surface_tilt, 'aspect': surface_azimuth,
              'pvcalculation': int(pvcalculation),
              'pvtechchoice': pvtechchoice, 'mountingplace': mountingplace,
              'trackingtype': trackingtype, 'components': int(components),
              'usehorizon': int(usehorizon),
              'optimalangles': int(optimalangles),
              'optimalinclination': int(optimal_surface_tilt), 'loss': loss}
    # pvgis only takes 0 for False, and 1 for True, not strings
    if userhorizon is not None:
        params['userhorizon'] = ','.join(str(x) for x in userhorizon)
    if raddatabase is not None:
        params['raddatabase'] = raddatabase
    if start is not None:
        params['startyear'] = start if isinstance(start, int) else start.year
    if end is not None:
        params['endyear'] = end if isinstance(end, int) else end.year
    if peakpower is not None:
        params['peakpower'] = peakpower
    
    res = requests.get(url + 'seriescalc', params=params, timeout=timeout)   
    if not res.ok:
        try:
            err_msg = res.json()
        except Exception:
            res.raise_for_status()
        else:
            raise requests.HTTPError(err_msg['message'])

    return read_pvgis_hourly(io.StringIO(res.text), pvgis_format=outputformat,
                             map_variables=map_variables)

def _parse_pvgis_hourly_json(src, map_variables):
    inputs = src['inputs']
    metadata = src['meta']
    data = pd.DataFrame(src['outputs']['hourly'])
    data.index = pd.to_datetime(data['time'], format='%Y%m%d:%H%M', utc=True)
    data = data.drop('time', axis=1)
    data = data.astype(dtype={'Int': 'int'})  # The 'Int' column to be integer
    if map_variables:
        data = data.rename(columns=VARIABLE_MAP)
    return data, inputs, metadata


def _parse_pvgis_hourly_csv(src, map_variables):
    # The first 4 rows are latitude, longitude, elevation, radiation database
    inputs = {}
    # 'Latitude (decimal degrees): 45.000\r\n'
    inputs['latitude'] = float(src.readline().split(':')[1])
    # 'Longitude (decimal degrees): 8.000\r\n'
    inputs['longitude'] = float(src.readline().split(':')[1])
    # Elevation (m): 1389.0\r\n
    inputs['elevation'] = float(src.readline().split(':')[1])
    # 'Radiation database: \tPVGIS-SARAH\r\n'
    inputs['radiation_database'] = src.readline().split(':')[1].strip()
    # Parse through the remaining metadata section (the number of lines for
    # this section depends on the requested parameters)
    while True:
        line = src.readline()
        if line.startswith('time,'):              
            names = line.strip().split(',')
            break       
        elif line.strip() != '':
            inputs[line.split(':')[0]] = line.split(':')[1].strip()
        elif line == '':  # If end of file is reached
            raise ValueError('No data section was detected. File has probably '
                             'been modified since being downloaded from PVGIS')    
    data_lines = []
    while True:
        line = src.readline()
        if line.strip() == '':
            break
        else:
            data_lines.append(line.strip().split(','))
    data = pd.DataFrame(data_lines, columns=names)
    data.index = pd.to_datetime(data['time'], format='%Y%m%d:%H%M', utc=True)
    data = data.drop('time', axis=1)
    if map_variables:
        data = data.rename(columns=VARIABLE_MAP) 
    data = data.astype(float).astype(dtype={'Int': 'int'})
    metadata = {}
    for line in src.readlines():
        if ':' in line:
            metadata[line.split(':')[0]] = line.split(':')[1].strip()
    return data, inputs, metadata

def read_pvgis_hourly(filename, pvgis_format=None, map_variables=True):
   
    # get the PVGIS outputformat
    if pvgis_format is None:
        outputformat = Path(filename).suffix[1:].lower()
    else:
        outputformat = pvgis_format

    if outputformat == 'json':
        try:
            src = json.load(filename)
        except AttributeError:  # str/path has no .read() attribute
            with open(str(filename), 'r') as fbuf:
                src = json.load(fbuf)
        return _parse_pvgis_hourly_json(src, map_variables=map_variables)

    if outputformat == 'csv':
        try:
            pvgis_data = _parse_pvgis_hourly_csv(
                filename, map_variables=map_variables)
        except AttributeError:  # str/path has no .read() attribute
            with open(str(filename), 'r') as fbuf:
                pvgis_data = _parse_pvgis_hourly_csv(
                    fbuf, map_variables=map_variables)
        return pvgis_data

    # raise exception if pvgis format isn't in ['csv', 'json']
    err_msg = (
        "pvgis format '{:s}' was unknown, must be either 'json' or 'csv'")\
        .format(outputformat)
    raise ValueError(err_msg)


In [339]:
#Defining the inputs
lat = 52.5 
lon = 13.4
start=pd.Timestamp('2006-12-01')
end=pd.Timestamp('2016-01-31')
raddatabase='PVGIS-SARAH2'
loss=15

data = get_pvgis_hourly(latitude = lat, 
                    longitude = lon, 
                    start=start, 
                    end=end,
                    raddatabase=raddatabase,
                    components=True, #Maybe need to change for 1
                    surface_tilt = pv_tilt, #this var comes from getting solar position part
                    surface_azimuth=0, # 0=south For Noth Hemisphere
                    outputformat='csv',
                    usehorizon=True, #Include effects of horizon
                    userhorizon=None, #Optional user specified elevation of horizon in degrees - Will not use it 
                    pvcalculation=True,
                    peakpower= peakpowervar, # Source: https://www.berlin.de/umweltatlas/en/energy/solar-systems/continually-updated/map-description/ 
                    pvtechchoice='crystSi',
                    mountingplace='free',  #Type of mounting for PV system. Options of 'free' for free-standing 
                                           #and 'building' for building-integrated.
                    loss= loss, #considering a global loss of 30% section 11.2.2 Source: https://www.ise.fraunhofer.de/content/dam/ise/en/documents/publications/studies/recent-facts-about-photovoltaics-in-germany.pdf 
                    trackingtype=0, #Type of mounting 0=fixed
                    optimal_surface_tilt=True, #Calculate the optimum tilt angle.
                    optimalangles=False, #Calculate the optimum tilt and azimuth angles.                      
                    url=URL, 
                    map_variables=True, # When true, renames columns of the Dataframe to pvlib variable names
                    timeout=30) # Time in seconds to wait for server response before timeout
PVgis_data = data[0]

## The idea is to make the mean of a hourly measure to create a var to join the 2 df

In [340]:
pvgis_temp = (
PVgis_data
    .reset_index()
    .assign(
        date = lambda x: x["time"].dt.date,
        m = lambda x: x["time"].dt.hour,
        ymdh = lambda x: x["date"].astype(str) + " " + x["m"].astype(str)
    )
)
#pvgis_temp.head()

In [341]:
pv_berlintemp = (
pv_berlin
    .reset_index()
    .assign(
        date = lambda x: x["time(UTC)"].dt.date,
        m = lambda x: x["time(UTC)"].dt.hour,
        ymdh = lambda x: x["date"].astype(str) + " " + x["m"].astype(str)
            )

)
pv_final = pd.merge(pv_berlintemp,pvgis_temp[["ymdh","P"]], on="ymdh", how="left")
#pv_final.info()

In [342]:
#creating a  meteorological season variable
pv_final.reset_index()
pv_final['season'] = pv_final["time(UTC)"].dt.month%12 // 3 + 1
#pv_final.head(3)

# Exploring the dataset

In [343]:
#saving the dataframe and importing it
#pv_final.to_csv("./pv_final.csv",index=False)
pv_final = pd.read_csv("pv_final.csv")
pv_final['time(UTC)'] = pd.to_datetime(pv_final['time(UTC)'])
# Checking for Missing Values and type of values of the columns.

In [344]:
pv_final.rename(columns={"P":"Power(Wh)"}, inplace=True)
pv_final.head()

,time(UTC),temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure,...,ac_current,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx,Power(Wh),season
0,2016-01-01 00:00:00+00:00,2.76,86.92,0.0,0.0,0.0,273.70,4.21,227.0,102055.0,...,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,2016-01-01 01:00:00+00:00,2.04,88.54,0.0,0.0,0.0,274.88,3.75,231.0,102065.0,...,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2016-01-01 02:00:00+00:00,1.32,90.16,0.0,0.0,0.0,276.05,3.29,234.0,102075.0,...,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,2016-01-01 03:00:00+00:00,0.60,91.78,0.0,0.0,0.0,277.23,2.83,238.0,102065.0,...,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,2016-01-01 04:00:00+00:00,-0.11,93.40,0.0,0.0,0.0,278.41,2.38,242.0,102055.0,...,-0.075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


### GRAPHS

In [345]:
import matplotlib.pyplot as plt
'''
column_plot =['temp_air', 'relative_humidity', 'ghi', 'dni', 'dhi',
       'IR(h)', 'wind_speed', 'wind_direction', 'pressure', 'apparent_zenith',
       'zenith', 'apparent_elevation', 'elevation', 'azimuth',
       'equation_of_time', 'dni_PVlib', 'aoi', 'poa_global', 'poa_direct',
       'poa_diffuse', 'poa_sky_diffuse', 'poa_ground_diffuse',
       'cell_temperature', 'effective_irradiance', 'ac_current', 'i_sc',
       'i_mp', 'v_oc', 'v_mp', 'p_mp', 'i_x', 'i_xx', "Power(Wh)"]

for column in column_plot:
    plt.figure()
    plt.figure(figsize = (16,8))
    ax = sns.histplot(data=pv_final[pv_final[column] != 0], x=column, kde=True, bins= 100) 
    splitcolumn = column.split("_")
    title = ""
    for name in splitcolumn:
        x_title = name.title()
        title += x_title + " " 
    ax.set_title(title + " Values", fontsize=18)
    title = ""
        
    ax = ax
    plt.show
'''

'\ncolumn_plot =[\'temp_air\', \'relative_humidity\', \'ghi\', \'dni\', \'dhi\',\n       \'IR(h)\', \'wind_speed\', \'wind_direction\', \'pressure\', \'apparent_zenith\',\n       \'zenith\', \'apparent_elevation\', \'elevation\', \'azimuth\',\n       \'equation_of_time\', \'dni_PVlib\', \'aoi\', \'poa_global\', \'poa_direct\',\n       \'poa_diffuse\', \'poa_sky_diffuse\', \'poa_ground_diffuse\',\n       \'cell_temperature\', \'effective_irradiance\', \'ac_current\', \'i_sc\',\n       \'i_mp\', \'v_oc\', \'v_mp\', \'p_mp\', \'i_x\', \'i_xx\', "Power(Wh)"]\n\nfor column in column_plot:\n    plt.figure()\n    plt.figure(figsize = (16,8))\n    ax = sns.histplot(data=pv_final[pv_final[column] != 0], x=column, kde=True, bins= 100) \n    splitcolumn = column.split("_")\n    title = ""\n    for name in splitcolumn:\n        x_title = name.title()\n        title += x_title + " " \n    ax.set_title(title + " Values", fontsize=18)\n    title = ""\n        \n    ax = ax\n    plt.show\n'

## Correlation

In [346]:
'''
# Defining the function to plot the correlation
def corr_graph(df):
    plt.figure(figsize=(30,15))
    mask = np.triu(np.ones_like(df.corr(), dtype=bool)) 
    heatmap = sns.heatmap(df.corr(),mask=mask, vmin=-1, vmax=1, annot=True, cmap="BrBG", fmt='.2f')
    heatmap.set_title("Correlation", fontdict={"fontsize":18}, pad=16)
    plt.show
    return(heatmap)

#calling the function
# wx, gtk, osx, tk, empty uses default
%matplotlib inline
#corr_graph(pv_final)
'''    

'\n# Defining the function to plot the correlation\ndef corr_graph(df):\n    plt.figure(figsize=(30,15))\n    mask = np.triu(np.ones_like(df.corr(), dtype=bool)) \n    heatmap = sns.heatmap(df.corr(),mask=mask, vmin=-1, vmax=1, annot=True, cmap="BrBG", fmt=\'.2f\')\n    heatmap.set_title("Correlation", fontdict={"fontsize":18}, pad=16)\n    plt.show\n    return(heatmap)\n\n#calling the function\n# wx, gtk, osx, tk, empty uses default\n%matplotlib inline\n#corr_graph(pv_final)\n'

# Making the first Interaction with ML

### The Power(KWh) column Power(KWh) is the variable to be predicted.

Need to create some features using datetime, for example day, weekday, month and year. And then remove the variable daytime from the Dataframe, because when I do the model trining the hot encoder works on the datetime and messes up with my results.  

In [347]:
# creating the day variable
pv_final['day'] = pv_final["time(UTC)"].dt.day
#pv_final['weekday'] = pv_final["time(UTC)"].dt.weekday
pv_final['weekday_name'] = pv_final["time(UTC)"].dt.day_name()
#pv_final['month'] =pv_final["time(UTC)"].dt.month
pv_final['month_name'] =pv_final["time(UTC)"].dt.month_name()
pv_final['year'] =pv_final["time(UTC)"].dt.year
pv_final.head(2)

,time(UTC),temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure,...,v_mp,p_mp,i_x,i_xx,Power(Wh),season,day,weekday_name,month_name,year
0,2016-01-01 00:00:00+00:00,2.76,86.92,0.0,0.0,0.0,273.70,4.21,227.0,102055.0,...,0.0,0.0,0.0,0.0,0.0,1,1,Friday,January,2016
1,2016-01-01 01:00:00+00:00,2.04,88.54,0.0,0.0,0.0,274.88,3.75,231.0,102065.0,...,0.0,0.0,0.0,0.0,0.0,1,1,Friday,January,2016


In [348]:
# Now dropping the time(UTC) column
pv_final.drop(columns=["time(UTC)"], inplace=True)
#pv_final.drop(columns=["time(UTC)"])
#pv_final.head(2)

In [349]:
# Removing the values = 0
pv_final_cleaned = pv_final.loc[~(pv_final["Power(Wh)"]==0)]
(pv_final_cleaned == 0).all()

temp_air                False
relative_humidity       False
ghi                     False
dni                     False
dhi                     False
IR(h)                   False
wind_speed              False
wind_direction          False
pressure                False
apparent_zenith         False
zenith                  False
apparent_elevation      False
elevation               False
azimuth                 False
equation_of_time        False
aoi                     False
poa_global              False
poa_direct              False
poa_diffuse             False
poa_sky_diffuse         False
poa_ground_diffuse      False
cell_temperature        False
effective_irradiance    False
ac_current              False
i_sc                    False
i_mp                    False
v_oc                    False
v_mp                    False
p_mp                    False
i_x                     False
i_xx                    False
Power(Wh)               False
season                  False
day       

### Training the model 

In [350]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 

y=  pv_final_cleaned["Power(Wh)"]
X = pv_final_cleaned
X = pv_final_cleaned.drop(columns="Power(Wh)")

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1990)

## Tuning the RidgeCV Linear model

In [351]:
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler

X_cat_columns = X_train.select_dtypes(include="object").columns
X_num_columns = X_train.select_dtypes(exclude="object").columns

#Setting the imputers, Scaler 
imputer = KNNImputer()
scaler = MinMaxScaler(feature_range=(0, 10))

# create numerical pipeline
numeric_pipe = make_pipeline(imputer,
                     scaler)
                     
# create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder()
)

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_columns),
        ("cat_pipe", categoric_pipe, X_cat_columns)  
    ]
)

from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor, 
                              RidgeCV())

param_grid = {    
    "columntransformer__cat_pipe__onehotencoder__handle_unknown" : ["ignore"],
    "ridgecv__alphas":(0.1, 2.0, 20.0),    
    "ridgecv__gcv_mode":[None, "auto", "svd", "eigen"],
    "ridgecv__fit_intercept":[True,False],
}

lm_search = RandomizedSearchCV(full_pipeline,
                      param_grid,
                      cv=10,
                      n_jobs=-1,
                      verbose=1)

lm_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('num_pipe',
                                                                               Pipeline(steps=[('knnimputer',
                                                                                                KNNImputer()),
                                                                                               ('minmaxscaler',
                                                                                                MinMaxScaler(feature_range=(0,
                                                                                                                            10)))]),
                                                                               Index(['temp_air', 'relative_humidity', 'ghi', 'dni', 'dhi', 'IR(h)',
       'wind_speed', 'wind_direction', 'pressure', 'apparent_zenith', 'zenith',
       'app...
                                                                                                              strategy='constant')),
                                                                                               ('onehotencoder',
                                                                                                OneHotEncoder())]),
                                                                               Index(['weekday_name', 'month_name'], dtype='object'))])),
                                             ('ridgecv', RidgeCV())]),
                   n_jobs=-1,
                   param_distributions={'columntransformer__cat_pipe__onehotencoder__handle_unknown': ['ignore'],
                                        'ridgecv__alphas': (0.1, 2.0, 20.0),
                                        'ridgecv__fit_intercept': [True, False],
                                        'ridgecv__gcv_mode': [None, 'auto',
                                                              'svd', 'eigen']},
                   verbose=1)

In [352]:
print(f'model score on training data: {lm_search.score(X_train, y_train).round(3)}')
print(f'model score on testing data: {lm_search.score(X_test, y_test).round(3)}')

model score on training data: 0.743
model score on testing data: 0.722


### Creating a function to plot a Histogram of the Predicted values 

In [353]:
'''
def Pred_graph(Y_search):
    plt.figure(figsize = (16,8))
    counts, bins = np.histogram(Y_search)
    ax = plt.hist(bins[:-1], bins, weights=counts)
    plt.ylabel('Number of Observations')
    plt.xlabel( "Power (Wh)")
    ax = ax
    plt.show
    return ax
'''

'\ndef Pred_graph(Y_search):\n    plt.figure(figsize = (16,8))\n    counts, bins = np.histogram(Y_search)\n    ax = plt.hist(bins[:-1], bins, weights=counts)\n    plt.ylabel(\'Number of Observations\')\n    plt.xlabel( "Power (Wh)")\n    ax = ax\n    plt.show\n    return ax\n'

In [359]:
test = pd.read_csv("pv_base.csv")
test.head(3)

,temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure,apparent_zenith,...,i_x,i_xx,Power(Wh),season,day,weekday,weekday_name,month,month_name,year
0,-0.20,94.53,30.0,170.05,15.0,268.15,1.66,251.0,102144.0,85.862670,...,0.054036,0.040079,2552.4,1,1,4,Friday,1,January,2016
1,0.99,93.21,54.0,152.85,27.0,265.10,1.67,254.0,102134.0,80.546226,...,0.112267,0.083135,1860.0,1,1,4,Friday,1,January,2016
2,2.17,91.90,18.0,38.81,9.0,262.05,1.69,258.0,102124.0,76.971560,...,0.040401,0.029978,2749.8,1,1,4,Friday,1,January,2016


In [363]:
test.index

RangeIndex(start=0, stop=4123, step=1)

In [355]:
'''
lm_pred = lm_search.predict(test)
y_predict = lm_search.predict(test)
y_predict = pd.DataFrame(data=y_predict)
y_predict.reset_index()
Pred_graph(y_predict)
'''

'\nlm_pred = lm_search.predict(test)\ny_predict = lm_search.predict(test)\ny_predict = pd.DataFrame(data=y_predict)\ny_predict.reset_index()\nPred_graph(y_predict)\n'

In [356]:
import pickle

#Linear Model
output_lm = open('./model9_hannover10peak.sav', "wb")
pickle.dump(lm_search, output_lm)
print("Model exported")

Model exported
